In [1]:
# 下载数据集
import torchvision
torchvision.datasets.MNIST('./data', download=True)

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train

In [7]:
# 搭建模型
import torch
import torch.nn as nn

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet,self).__init__()
        self.C1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=2)
        self.R1 = nn.ReLU()
        self.S2 = nn.MaxPool2d(kernel_size=2)
        self.C3 = nn.Conv2d(6, 16, 5, 1, 0)
        self.R2 = nn.ReLU()
        self.S4 = nn.MaxPool2d(2)
        self.C5 = nn.Conv2d(16, 120, 5, 1, 0)
        self.R3 = nn.ReLU()
        self.F6 = nn.Linear(in_features=120, out_features=84)
        self.R4 = nn.ReLU()
        self.OUT = nn.Linear(84, 10)
        # 向前传播函数
    def forward(self, x):
        x = self.C1(x)
        x = self.R1(x)
        x = self.S2(x)
        x = self.C3(x)
        x = self.R2(x)
        x = self.S4(x)
        x = self.C5(x)
        x = self.R3(x)
        x = x.view(x.size(0), -1)
        x = self.F6(x)
        x = self.R4(x)
        x = self.OUT(x)
        return x

In [8]:
# 训练模型
import torchvision
import torch.utils.data as Data

#导入模型
model=LeNet()

#遍历五轮，批大小设置为64，初始学习率为0.001
Epoch = 5
batch_size = 64
lr = 0.001

#导入数据集
train_data = torchvision.datasets.MNIST(root='./data/', train=True, transform=torchvision.transforms.ToTensor(), download=False)
#训练导入
train_loader = Data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=0, drop_last=True)
#损失函数和优化器
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

torch.set_grad_enabled(True)
model.train()

# 使用cuda
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

LeNet(
  (C1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (R1): ReLU()
  (S2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (C3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (R2): ReLU()
  (S4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (C5): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  (R3): ReLU()
  (F6): Linear(in_features=120, out_features=84, bias=True)
  (R4): ReLU()
  (OUT): Linear(in_features=84, out_features=10, bias=True)
)

In [9]:
# 训练
for epoch in range(Epoch):
    running_loss = 0.0
    acc = 0.0
    for step, data in enumerate(train_loader):
        x, y = data
        optimizer.zero_grad()
        y_pred = model(x.to(device, torch.float))
        loss = loss_function(y_pred, y.to(device, torch.long))
        loss.backward()
        running_loss += float(loss.data.cpu())
        pred = y_pred.argmax(dim=1)
        acc += (pred.data.cpu() == y.data).sum()
        optimizer.step()
        if step % 100 == 99:
            loss_avg = running_loss / (step + 1)
            acc_avg = float(acc / ((step + 1) * batch_size))
            print('Epoch', epoch + 1, ',step', step + 1, '| Loss_avg: %.4f' % loss_avg, '|Acc_avg:%.4f' % acc_avg)

Epoch 1 ,step 100 | Loss_avg: 1.2090 |Acc_avg:0.6328
Epoch 1 ,step 200 | Loss_avg: 0.8027 |Acc_avg:0.7538
Epoch 1 ,step 300 | Loss_avg: 0.6292 |Acc_avg:0.8070
Epoch 1 ,step 400 | Loss_avg: 0.5281 |Acc_avg:0.8386
Epoch 1 ,step 500 | Loss_avg: 0.4608 |Acc_avg:0.8595
Epoch 1 ,step 600 | Loss_avg: 0.4087 |Acc_avg:0.8749
Epoch 1 ,step 700 | Loss_avg: 0.3720 |Acc_avg:0.8858
Epoch 1 ,step 800 | Loss_avg: 0.3404 |Acc_avg:0.8953
Epoch 1 ,step 900 | Loss_avg: 0.3162 |Acc_avg:0.9030
Epoch 2 ,step 100 | Loss_avg: 0.1006 |Acc_avg:0.9716
Epoch 2 ,step 200 | Loss_avg: 0.0961 |Acc_avg:0.9708
Epoch 2 ,step 300 | Loss_avg: 0.0913 |Acc_avg:0.9714
Epoch 2 ,step 400 | Loss_avg: 0.0897 |Acc_avg:0.9714
Epoch 2 ,step 500 | Loss_avg: 0.0876 |Acc_avg:0.9722
Epoch 2 ,step 600 | Loss_avg: 0.0837 |Acc_avg:0.9736
Epoch 2 ,step 700 | Loss_avg: 0.0838 |Acc_avg:0.9739
Epoch 2 ,step 800 | Loss_avg: 0.0814 |Acc_avg:0.9745
Epoch 2 ,step 900 | Loss_avg: 0.0802 |Acc_avg:0.9749
Epoch 3 ,step 100 | Loss_avg: 0.0687 |Acc_avg:

In [10]:
# 模型保存
torch.save(model, './LeNet.pkl')

In [11]:
# 测试模型
import torch
import torchvision
import torch.utils.data as Data

test_data = torchvision.datasets.MNIST(root='./data/', train=False, transform=torchvision.transforms.ToTensor(), download=False)
test_loader = Data.DataLoader(test_data, batch_size=1, shuffle=False)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = torch.load('./LeNet.pkl',map_location=torch.device(device))
net.to(device)
torch.set_grad_enabled(False)
net.eval()
length = test_data.data.size(0)
acc = 0.0

for i, data in enumerate(test_loader):
    x, y = data
    y_pred = net(x.to(device, torch.float))
    pred = y_pred.argmax(dim=1)
    acc += (pred.data.cpu() == y.data).sum()
    print('Predict:', int(pred.data.cpu()), '|Ground Truth:', int(y.data))
acc = (acc / length) * 100
print('Accuracy: %.2f' %acc, '%')

Predict: 7 |Ground Truth: 7
Predict: 2 |Ground Truth: 2
Predict: 1 |Ground Truth: 1
Predict: 0 |Ground Truth: 0
Predict: 4 |Ground Truth: 4
Predict: 1 |Ground Truth: 1
Predict: 4 |Ground Truth: 4
Predict: 9 |Ground Truth: 9
Predict: 5 |Ground Truth: 5
Predict: 9 |Ground Truth: 9
Predict: 0 |Ground Truth: 0
Predict: 6 |Ground Truth: 6
Predict: 9 |Ground Truth: 9
Predict: 0 |Ground Truth: 0
Predict: 1 |Ground Truth: 1
Predict: 5 |Ground Truth: 5
Predict: 9 |Ground Truth: 9
Predict: 7 |Ground Truth: 7
Predict: 3 |Ground Truth: 3
Predict: 4 |Ground Truth: 4
Predict: 9 |Ground Truth: 9
Predict: 6 |Ground Truth: 6
Predict: 6 |Ground Truth: 6
Predict: 5 |Ground Truth: 5
Predict: 4 |Ground Truth: 4
Predict: 0 |Ground Truth: 0
Predict: 7 |Ground Truth: 7
Predict: 4 |Ground Truth: 4
Predict: 0 |Ground Truth: 0
Predict: 1 |Ground Truth: 1
Predict: 3 |Ground Truth: 3
Predict: 1 |Ground Truth: 1
Predict: 3 |Ground Truth: 3
Predict: 4 |Ground Truth: 4
Predict: 7 |Ground Truth: 7
Predict: 2 |Ground T